## Finetuning DistilBERT using SQuAD 2.0

This notebook contains the following fine-tuning groups: Model_19

....

The starting-point for code in this file was found in the Medium blog post titled, Question Answering with DistilBERT (https://medium.com/@sabrinaherbst/question-answering-with-distilbert-ba3e178fdf3d). Main differences include:

 - The DistilBERT model was pre-trined using SQuAD 2.0, rather than SQuAD 1.0
 - Exploring traditional split that included unseen test data  (i.e. not validation data)
 - Addition of adhoc dropout rate setting
 - Addition of alternate dataloader with Fixed-Length Truncation
 - Addition of performance analysis for each category

....

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load libraries

from transformers import DistilBertModel, DistilBertForMaskedLM, DistilBertConfig, \
            DistilBertTokenizerFast, AutoTokenizer, BertModel, BertForMaskedLM, BertTokenizerFast, BertConfig
from torch import nn
from pathlib import Path
import torch
import pandas as pd
from typing import Optional
from tqdm.auto import tqdm
from torch.optim import AdamW, RMSprop
import numpy as np

import sys
sys.path.append('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project')
from qa_model import QuestionDistilBERT, SimpleQuestionDistilBERT, ReuseQuestionDistilBERT, Dataset, test_model
from util import eval_test_set, count_parameters, print_test_set_incorrect_predictions, \
                 analyze_test_set_performance, eval_test_set_by_category, eval_test_set_pure, display_category_examples
from my_distilbert import QADataset

# Load tokenizer


In [ ]:
# Load DistilBERT tokenizer, use uncased (lowercase) vocabulary

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Load data_2 (Traditional split)

Note: QADataset loader is the Fixed-Length Trunc" described in the report

In [ ]:
# Get paths for all SQuAD dataset text files in training directory -- data_2, Traditional Split
squad_paths_2 = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')]

# Create training dataset using only SQuAD data, DataLoader with batch size of 8
dataset_2 = QADataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')],
                  natural_question_paths=None,
                  hotpotqa_paths=None, tokenizer=tokenizer)
loader_2 = torch.utils.data.DataLoader(dataset_2, batch_size=8)
print(f"Approximate Training Dataset Size: {len(dataset_2)}")

## load the validation dataset -- used to be labeled as "test", test_dataset changed to val_dataset, test_loader changed to val_loader
val_dataset_2 = QADataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/validation_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
val_loader_2 = torch.utils.data.DataLoader(val_dataset_2, batch_size=4)
print(f"Approximate Validation Dataset Size: {len(val_dataset_2)}")

## load the test dataset -- test_dataset and test_loader should not be used during training
test_dataset_2 = QADataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/test_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=4)
print(f"Approximate Test Dataset Size: {len(test_dataset_2)}")

Loaded 113506 total samples
Approximate Training Dataset Size: 113506
Loaded 14181 total samples
Approximate Validation Dataset Size: 14181
Loaded 14190 total samples
Approximate Test Dataset Size: 14190


# Model_19
Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split; Alter training parameters with 3 epochs, dropout rate of 0.13 and AdamW optimizer rather than RMSprop

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_19 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_19 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_19 = model_19.distilbert

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_19 = SimpleQuestionDistilBERT(mod_19)
model_19.set_dropout_rate(0.13)
model_19.to(device)

# Verify the dropout rates for each layer
for name, module in model_19.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.13
distilbert.transformer.layer.0.attention.dropout: 0.13
distilbert.transformer.layer.0.ffn.dropout: 0.13
distilbert.transformer.layer.1.attention.dropout: 0.13
distilbert.transformer.layer.1.ffn.dropout: 0.13
distilbert.transformer.layer.2.attention.dropout: 0.13
distilbert.transformer.layer.2.ffn.dropout: 0.13
distilbert.transformer.layer.3.attention.dropout: 0.13
distilbert.transformer.layer.3.ffn.dropout: 0.13
distilbert.transformer.layer.4.attention.dropout: 0.13
distilbert.transformer.layer.4.ffn.dropout: 0.13
distilbert.transformer.layer.5.attention.dropout: 0.13
distilbert.transformer.layer.5.ffn.dropout: 0.13
dropout: 0.13


In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_19.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = AdamW(model_19.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 3 epochs
# Validation data used to evaluate performance during training

epochs = 4

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_19.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_19(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_19.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_19(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14189 [00:00<?, ?it/s]

Mean Training Loss 3.540464662404534


  0%|          | 0/3546 [00:00<?, ?it/s]

Mean Validation Loss 3.2170208388203054


  0%|          | 0/14189 [00:00<?, ?it/s]

...

Mean Validation Loss 2.881068672498954


  0%|          | 0/14189 [00:00<?, ?it/s]

Mean Training Loss 2.42897998969914


  0%|          | 0/3546 [00:00<?, ?it/s]

Mean Validation Loss 2.887486028941863


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_19.state_dict(), "simple_distilbert_qa_data_19.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_19 = SimpleQuestionDistilBERT(mod_19)

# Load previously saved model parameters from disk
model_19.load_state_dict(torch.load("simple_distilbert_qa_data_19.model"))

<ipython-input-11-2de2369c067b>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_19.load_state_dict(torch.load("simple_distilbert_qa_data_19.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_19, tokenizer, test_loader_2, device)

100%|██████████| 3548/3548 [01:05<00:00, 53.98it/s]

Mean EM:  0.7615221987315011
Mean F-1:  0.813053208354688


In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set_pure(model_19, tokenizer, test_loader_2, device)

100%|██████████| 3548/3548 [01:04<00:00, 54.77it/s]

Mean Pure EM:  0.758985200845666
Mean Pure F-1:  0.8100452661492713


In [ ]:
# Then add category evaluation
print("\nAnalyzing performance by question category:")
eval_test_set_by_category(model_19, tokenizer, test_loader_2, device)


Analyzing performance by question category:


100%|██████████| 3548/3548 [01:05<00:00, 54.46it/s]

+---------------+-------+------------+---------+---------+
| Question Type | Count | % of Total | Mean EM | Mean F1 |
+---------------+-------+------------+---------+---------+
|      what     |  8188 |   57.7%    |  0.769  |  0.817  |
|      when     |  958  |    6.8%    |  0.719  |  0.779  |
|      how      |  1422 |   10.0%    |  0.798  |  0.839  |
|      who      |  1469 |   10.4%    |  0.733  |  0.802  |
|      why      |  179  |    1.3%    |  0.810  |  0.839  |
|     other     |  283  |    2.0%    |  0.749  |  0.805  |
|     where     |  627  |    4.4%    |  0.738  |  0.803  |
|     which     |  1064 |    7.5%    |  0.735  |  0.795  |
+---------------+-------+------------+---------+---------+


{'what': {'count': 8188,
  'percentage': 57.70260747004933,
  'mean_em': 0.7686858817782121,
  'mean_f1': 0.8168330004599124},
 'when': {'count': 958,
  'percentage': 6.75123326286117,
  'mean_em': 0.7192066805845512,
  'mean_f1': 0.779205364225046},
 'how': {'count': 1422,
  'percentage': 10.021141649048626,
  'mean_em': 0.7981715893108298,
  'mean_f1': 0.8391682762932133},
 'who': {'count': 1469,
  'percentage': 10.352360817477097,
  'mean_em': 0.7331518039482642,
  'mean_f1': 0.8020993653372934},
 'why': {'count': 179,
  'percentage': 1.2614517265680056,
  'mean_em': 0.8100558659217877,
  'mean_f1': 0.8387357743532653},
 'other': {'count': 283,
  'percentage': 1.9943622269203665,
  'mean_em': 0.7491166077738516,
  'mean_f1': 0.8045547250780034},
 'where': {'count': 627,
  'percentage': 4.4186046511627906,
  'mean_em': 0.7384370015948963,
  'mean_f1': 0.8026849420487482},
 'which': {'count': 1064,
  'percentage': 7.498238195912614,
  'mean_em': 0.7349624060150376,
  'mean_f1': 0.7948

In [ ]:
# Get examples by category
examples = display_category_examples(model_19, tokenizer, test_loader_2, device)

100%|██████████| 3548/3548 [01:04<00:00, 54.66it/s]


+---------------+-------+------------+---------+---------+
| Question Type | Count | % of Total | Mean EM | Mean F1 |
+---------------+-------+------------+---------+---------+
|      what     |  8188 |   57.7%    |  0.769  |  0.817  |
|      when     |  958  |    6.8%    |  0.719  |  0.779  |
|      how      |  1422 |   10.0%    |  0.798  |  0.839  |
|      who      |  1469 |   10.4%    |  0.733  |  0.802  |
|      why      |  179  |    1.3%    |  0.810  |  0.839  |
|     other     |  283  |    2.0%    |  0.749  |  0.805  |
|     where     |  627  |    4.4%    |  0.738  |  0.803  |
|     which     |  1064 |    7.5%    |  0.735  |  0.795  |
+---------------+-------+------------+---------+---------+


100%|██████████| 3548/3548 [01:04<00:00, 54.62it/s]



==================== WHAT Questions ====================

✅ Successful Examples (EM=1):

1. Question: what ' s another thing the paper showed hostility to? despite its initial opposition to the closures, until 1997, the newspaper repeatedly called for the implementation of further thatcherite policies, such as royal mail privatisation, [ verification needed ] and social security cutbacks, with leaders such as " peter lilley is right, we can ' t carry on like this ", [ verification needed ] the paper showed hostility to the eu and approval of public spending cuts, tax cuts, and promotion of right - wing ministers to the cabinet, with leaders such as " more of the redwood, not deadwood ".
   Predicted: 
   Truth: 

2. Question: what type of technology is used to connect to the internet wirelessly? isps provide internet access, employing a range of technologies to connect users to their network. available technologies have ranged from computer modems with acoustic couplers to telephone l